<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Escribir csv
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 31/05/2024 | v1r0        | Primera versión del cuaderno. |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#instanciacion)
3. [Creación de los atributos a partir de las métricas](#creacion)
4. [Escritura del csv](#escritura)

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuadernillo se encuentra todo el codigo en general que hemos utilizado para escribir el csv. Este csv es el paso previo necesario para evaluar y entrenar los modelos con los atributos seleccionados (centralidades, clustering y comunidades).

<br></br>

# <a name="instanciacion"></a> 2. **Instanciación del grafo**

<br></br>

In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import json
import csv

# Step 1: Read the edges.csv file
with open('../facebook_large/musae_facebook_edges.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    edges = [tuple(row) for row in edges_data]

# Step 3: Create a NetworkX graph object
G = nx.Graph()

# Step 4: Add nodes and edges to the graph
G.add_edges_from(edges)

tvshow = []
government = []
politician = []
company = []

# Step 5: Assign features to nodes or edges
with open('../facebook_large/musae_facebook_target.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    for row in edges_data:
        target = row[-1]
        node = row[0]
        if target == "government":
            government.append(node)
        elif target == "tvshow":
            tvshow.append(node)
        elif target == "politician":
            politician.append(node)
        elif target == "company":
            company.append(node)
            
        G.nodes[node]["target"] = target



# <a name="creacion"></a> 3. **Creación de los atributos a partir de las métricas**

<br></br>

In [3]:
from itertools import permutations

# Calcular k-path centrality
def k_path_centrality(G, k):
    # Inicializar el diccionario para almacenar la centralidad
    centrality = {node: 0 for node in G.nodes()}
    
    # Generar todos los k-paths posibles en el grafo
    for path in permutations(G.nodes(), k + 1):
        # Verificar si es un camino válido (todos los nodos consecutivos están conectados)
        valid_path = all(G.has_edge(path[i], path[i+1]) for i in range(k))
        if valid_path:
            # Incrementar el contador para cada nodo en el camino
            for node in path:
                centrality[node] += 1
                
    # Normalizar la centralidad dividiendo por el número total de caminos generados
    total_paths = sum(centrality.values())
    if total_paths > 0:
        for node in centrality:
            centrality[node] /= total_paths
            
    return centrality

In [4]:
def communitie_to_id(communities,G):
    ids = dict()
    for node in G.nodes:
        for id in range(len(communities)):
            if node in communities[id]:
                ids[node] = id
                break
    return ids

def communitie_to_id2(communities,G):
    communities = list(communities)
    ids = dict()
    for id in range(len(communities)):
        for node in communities[id]:
            if node in ids:
                print("ERROR")
            ids[node] = id
    return ids

In [5]:
from networkx.algorithms import community

greedy_modularity_communities = community.greedy_modularity_communities(G)
greedy_modularity_communities_nodes = communitie_to_id(greedy_modularity_communities, G)
# Centralidad de intermeciadión
betweenness_centrality = nx.betweenness_centrality(G)
# Centralidad de grado 
degree_centrality = nx.degree_centrality(G)
# Centralidad k-path (k=1)


In [ ]:
k_path_centrality = k_path_centrality(G, k=1)
# Centralidad de cercanía
closeness_centrality = nx.closeness_centrality(G)
# Centralidad harmonica
harmonic_centrality = nx.harmonic_centrality(G)
# Coeficiente de clustering
clustering = nx.clustering(G)

In [ ]:
square_clustering = nx.square_clustering(G)

In [ ]:
from networkx.algorithms import community
label_propagation_communities = community.label_propagation_communities(G)
label_propagation_communities_nodes = communitie_to_id2(label_propagation_communities,G)
label_propagation_communities_nodes

# <a name="escritura"></a> 3. **Escritura del csv**

<br></br>

In [ ]:
# Los calculos se hacen por separado para evitar que si hay algun error en la escritura que se tengan que rehacer
array_node_values = []
i = 0
with open('../facebook_large/musae_facebook_target.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    for row in edges_data:
        target = row[-1]
        node = row[0]

        
        values_to_append = []
        values_to_append.append(greedy_modularity_communities_nodes[node])
        values_to_append.append(label_propagation_communities_nodes[node])
        values_to_append.append(betweenness_centrality[node])
        values_to_append.append(degree_centrality[node])
        values_to_append.append(k_path_centrality[node])
        values_to_append.append(closeness_centrality[node])
        values_to_append.append(harmonic_centrality[node])
        values_to_append.append(square_clustering[node])
        values_to_append.append(clustering[node])
        values_to_append.append(target)

        i = i+1
        
        array_node_values.append(values_to_append)

with open("dict.csv","w") as df:
    df.write("greedy_modularity_communities_id,label_propagation_communities_id,betweenness_centrality,degree_centrality,k_path_centrality,closeness_centrality,harmonic_centrality,square_clustering,clustering,target\n")
    for line in array_node_values:
        str_to_write = ",".join(map(str,line))
        df.write(str_to_write+"\n")

    df.close()
            
        
        